In [1]:
import pandas as pd
import numpy as np
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

* Currently only get the fill limit value for each **numeric** data in chart_MV and chart_CV

In [4]:
mimic_chart_MV = pd.read_csv("./data/mimic_chart_MV_only_numeric.csv")
mimic_chart_CV = pd.read_csv("./data/mimic_chart_CV_only_numeric.csv")

/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


* Get fill_limit_value_chart_MV

In [10]:
temporal_vars_chart_mv = set(mimic_chart_MV["itemid"].unique())
itemid_value_counts_chart_mv = mimic_chart_MV[mimic_chart_MV["itemid"].isin(temporal_vars_chart_mv)]["itemid"].value_counts().rename_axis('itemid_chart_mv').reset_index(name='counts')
print(itemid_value_counts_chart_mv.shape)
final_itemid_value_counts_chart_mv = itemid_value_counts_chart_mv[itemid_value_counts_chart_mv["counts"] > 1]
print(final_itemid_value_counts_chart_mv.shape)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(final_itemid_value_counts_chart_mv)

(469, 2)
(426, 2)


,itemid_chart_mv,counts
0,220045,678005
1,220210,671724
2,220277,657704
3,220181,367134
4,220179,366629
5,220180,366468
6,220052,234758
7,220050,232992
8,220051,232932
9,220074,128695


In [11]:
itemid_fill_limit_chart_MV = pd.DataFrame(np.zeros((1, final_itemid_value_counts_chart_mv.shape[0])), columns=list(final_itemid_value_counts_chart_mv["itemid_chart_mv"].unique()))
itemid_fill_limit_chart_MV

,220045,220210,220277,220181,220179,220180,220052,220050,220051,220074,...,227776,227860,228162,226769,220283,227450,226983,227428,226754,226541
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
groupby_itemid_chart_MV = mimic_chart_MV.groupby("itemid")

In [12]:
for i, item_id in enumerate(final_itemid_value_counts_chart_mv["itemid_chart_mv"]):
    cur_itemid_group = groupby_itemid_chart_MV.get_group(item_id)
    cur_adm_ids_with_cur_itemid = cur_itemid_group["hadm_id"].unique()
    patient_group_for_cur_itemid = cur_itemid_group.groupby("hadm_id")
    cur_itemid_time_until_remeasure_list = []
    for j, hadm_id in enumerate(cur_adm_ids_with_cur_itemid):
        cur_patient_cur_itemid = patient_group_for_cur_itemid.get_group(hadm_id)
        if cur_patient_cur_itemid.shape[0] == 1: # current patient only has one value for cur_itemid
            continue
        else:
            cur_patient_cur_itemid["charttimestamp"] = cur_patient_cur_itemid[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"], "%Y-%m-%d %H:%M:%S"))), axis=1)
            cur_patient_cur_itemid.sort_values("charttimestamp", inplace=True)
            cur_patient_cur_itemid.reset_index(drop=True, inplace=True)
            initial_charttime = cur_patient_cur_itemid.iloc[0]["charttimestamp"]
            cur_patient_cur_itemid["interval_from_1st_record"] = ((cur_patient_cur_itemid["charttimestamp"]-initial_charttime) / 3600).astype(int)
            cur_patient_cur_itemid["interval_until_next_remeasure"] = cur_patient_cur_itemid["interval_from_1st_record"].diff()
            cur_itemid_time_until_remeasure_list += list(cur_patient_cur_itemid["interval_until_next_remeasure"].values[1:])
#             print(cur_itemid_time_until_remeasure_list)
#             display(cur_patient_cur_itemid)
    if len(cur_itemid_time_until_remeasure_list) == 0: # no patients record this itemid for multiple times
        itemid_fill_limit_chart_MV.loc[0][item_id] = 0
    else:
        itemid_fill_limit_chart_MV.loc[0][item_id] = np.quantile(cur_itemid_time_until_remeasure_list, 0.8)
    

In [14]:
fill_limit_chart_MV = itemid_fill_limit_chart_MV.T
fill_limit_chart_MV.reset_index(inplace=True)
fill_limit_chart_MV.rename(columns={"index": "itemid", 0: "recommend_fill_limit_MV"}, inplace=True)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(fill_limit_chart_MV)

,itemid,recommend_fill_limit_MV
0,220045,1.0
1,220210,1.0
2,220277,1.0
3,220181,1.0
4,220179,1.0
5,220180,1.0
6,220052,1.0
7,220050,1.0
8,220051,1.0
9,220074,1.0


In [15]:
fill_limit_chart_MV.to_csv("./data/fill_limit_chart_MV.csv", index=False)

* Get fill_limit_chart_CV

In [16]:
temporal_vars_chart_cv = set(mimic_chart_CV["itemid"].unique())
itemid_value_counts_chart_cv = mimic_chart_CV[mimic_chart_CV["itemid"].isin(temporal_vars_chart_cv)]["itemid"].value_counts().rename_axis('itemid_chart_cv').reset_index(name='counts')
print(itemid_value_counts_chart_cv.shape)
final_itemid_value_counts_chart_cv = itemid_value_counts_chart_cv[itemid_value_counts_chart_cv["counts"] > 1]
print(final_itemid_value_counts_chart_cv.shape)

(1089, 2)
(978, 2)


In [17]:
itemid_fill_limit_chart_CV = pd.DataFrame(np.zeros((1, final_itemid_value_counts_chart_cv.shape[0])), columns=list(final_itemid_value_counts_chart_cv["itemid_chart_cv"].unique()))
itemid_fill_limit_chart_CV

,211,742,618,646,51,8368,52,5815,455,8549,...,1975,2647,1716,34,132,1558,1559,1560,1561,2687
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
groupby_itemid_chart_CV = mimic_chart_CV.groupby("itemid")
for i, item_id in enumerate(final_itemid_value_counts_chart_cv["itemid_chart_cv"]):
    cur_itemid_group = groupby_itemid_chart_CV.get_group(item_id)
    cur_adm_ids_with_cur_itemid = cur_itemid_group["hadm_id"].unique()
    patient_group_for_cur_itemid = cur_itemid_group.groupby("hadm_id")
    cur_itemid_time_until_remeasure_list = []
    for j, hadm_id in enumerate(cur_adm_ids_with_cur_itemid):
        cur_patient_cur_itemid = patient_group_for_cur_itemid.get_group(hadm_id)
        if cur_patient_cur_itemid.shape[0] == 1: # current patient only has one value for cur_itemid
            continue
        else:
            cur_patient_cur_itemid["charttimestamp"] = cur_patient_cur_itemid[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"], "%Y-%m-%d %H:%M:%S"))), axis=1)
            cur_patient_cur_itemid.sort_values("charttimestamp", inplace=True)
            cur_patient_cur_itemid.reset_index(drop=True, inplace=True)
            initial_charttime = cur_patient_cur_itemid.iloc[0]["charttimestamp"]
            cur_patient_cur_itemid["interval_from_1st_record"] = ((cur_patient_cur_itemid["charttimestamp"]-initial_charttime) / 3600).astype(int)
            cur_patient_cur_itemid["interval_until_next_remeasure"] = cur_patient_cur_itemid["interval_from_1st_record"].diff()
            cur_itemid_time_until_remeasure_list += list(cur_patient_cur_itemid["interval_until_next_remeasure"].values[1:])
#             print(cur_itemid_time_until_remeasure_list)
#             display(cur_patient_cur_itemid)
    if len(cur_itemid_time_until_remeasure_list) == 0: # no patients record this itemid for multiple times
        itemid_fill_limit_chart_CV.loc[0][item_id] = 0
    else:
        itemid_fill_limit_chart_CV.loc[0][item_id] = np.quantile(cur_itemid_time_until_remeasure_list, 0.8)
    

In [19]:
fill_limit_chart_CV = itemid_fill_limit_chart_CV.T
fill_limit_chart_CV.reset_index(inplace=True)
fill_limit_chart_CV.rename(columns={"index": "itemid", 0: "recommend_fill_limit_CV"}, inplace=True)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(fill_limit_chart_CV)

,itemid,recommend_fill_limit_CV
0,211,1.0
1,742,1.0
2,618,1.0
3,646,1.0
4,51,1.0
5,8368,1.0
6,52,1.0
7,5815,1.0
8,455,1.0
9,8549,1.0


In [20]:
fill_limit_chart_CV.to_csv("./data/fill_limit_chart_CV.csv", index=False)